# Latent Dirichlet Allocation

<img src='https://miro.medium.com/max/1014/1*2uj6t3gNv76SpHrWf5-z-A.jpeg' img/>

<img src='https://miro.medium.com/max/2361/1*fCc0JT3W-1ViYyw0hJ7rdA.jpeg' img/>

### Resources: 

https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-latent-dirichlet-allocation-437c81220158

https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d

https://www.coursera.org/lecture/text-mining/3-9-latent-dirichlet-allocation-lda-part-1-deiXc

https://www.coursera.org/lecture/ml-clustering-and-retrieval/mixed-membership-models-for-documents-hQBJI

## Data

We will be using articles from NPR (National Public Radio), obtained from their website [www.npr.org](http://www.npr.org)

In [2]:
!pip install pyLDAvis

In [3]:
import pandas as pd

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

/Users/gabrielseemann/anaconda3/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


In [4]:
npr = pd.read_csv('npr.csv')

In [5]:
npr.shape

(11992, 1)

In [6]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Preprocessing

In [77]:
import re 
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def spacy_tokenizer(text):
    # remove html tags from all of the text before processing
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    #remove punctuation
#     cleanr2 = re.compile('_*')
#     cleantext2 = re.sub(cleanr2, '', cleantext)
    # Creating our token object, which is used to create documents with linguistic annotations.
    # we disabled the parser and ner parts of the pipeline in order to speed up parsing
    mytokens = nlp(cleantext, disable=['parser', 'ner'])

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [78]:
from sklearn.feature_extraction.text import CountVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [79]:
cv = CountVectorizer(tokenizer=spacy_tokenizer, max_df=0.90, min_df=10, stop_words='english', ngram_range=(1,3))

In [80]:
# df['num_legs'].sample(n=3, random_state=1)

dtm = cv.fit_transform(npr['Article'].sample(n=100, random_state=1))

In [2]:
dtm

NameError: name 'dtm' is not defined

## LDA

In [58]:
from sklearn.decomposition import LatentDirichletAllocation

In [59]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=20,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=20,
                          mean_change_tol=0.001, n_components=20, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [60]:
# This can take awhile, we're dealing with a large amount of documents!

lda_output = lda_model.fit_transform(dtm)


### Diagnose model performance with perplexity and log-likelihood
A model with higher log-likelihood and lower perplexity (exp(-1. * log-likelihood per word)) is considered to be good. Let’s check for our model.



In [61]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(dtm))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(dtm))

# See model parameters
print(lda_model.get_params())

Log Likelihood:  -108575.49819338245
Perplexity:  452.5338106040886
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'online', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 20, 'mean_change_tol': 0.001, 'n_components': 20, 'n_jobs': -1, 'perp_tol': 0.1, 'random_state': 100, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


## Showing Stored Words

In [62]:
len(cv.get_feature_names())

486

In [63]:
import random

In [64]:
for i in range(10):
    random_word_id = random.randint(0,99)
    print(cv.get_feature_names()[random_word_id])

better
able
benefit
board
appear
ago
age
base
..
big


In [65]:
for i in range(10):
    random_word_id = random.randint(0,99)
    print(cv.get_feature_names()[random_word_id])

clear
000
city
change
come
buy
accord
30
black
audience


### Showing Top Words Per Topic

In [66]:
len(lda_model.components_)

20

In [67]:
lda_model.components_

array([[ 0.11348128,  0.1160534 ,  0.12102347, ...,  0.11785489,
         6.70237968,  0.12534012],
       [ 0.11830155,  0.11406834,  0.10538468, ...,  0.11097975,
         0.13619928,  0.1120473 ],
       [ 0.15306633,  0.12281894,  1.06841359, ...,  0.13010307,
        11.12073653,  0.12380488],
       ...,
       [ 0.13193084,  0.11591592,  0.12413682, ...,  0.10782267,
         0.17657542,  0.11750954],
       [ 0.12041432,  0.11267751,  3.63652965, ...,  0.11619437,
         0.159876  ,  0.11088834],
       [ 2.03452539,  2.44572971,  0.17285811, ...,  0.10767103,
         6.56256098,  0.12346993]])

In [68]:
len(lda_model.components_[0])

486

In [69]:
single_topic = lda_model.components_[0]

In [70]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([308, 292, 315, 452, 299, 378, 300, 276, 224, 134, 398, 212, 329,
       397,  66, 428, 309,  38, 435,  55, 319,  43, 439, 340, 237, 105,
       274, 230, 446, 166, 117, 341, 165, 418, 355, 379, 471,  25,  93,
       221, 392, 245, 107, 297,  23, 106,  41, 337, 347, 426, 244, 338,
       145, 367, 360, 100, 470, 331, 187,  51, 208, 357, 316, 147,  61,
       227,  68, 420, 394, 376, 160, 266,  85,  18, 194,  16, 417, 243,
        56, 126, 135, 396, 216, 365, 348, 119, 356, 469, 441, 305, 197,
        47, 399,  33,  21, 429,  77, 407,   4,  90, 186, 238, 234, 323,
       436,  46,   0, 225, 205, 333, 204, 322, 359, 241, 257, 443, 321,
       143,  58,  75, 102, 130, 413, 451, 120, 124,  11, 167,  82, 104,
       157, 389,   5, 324, 387, 335, 363,  53,   1, 401,   7, 118, 421,
        54,  97,  48, 334, 342, 383, 152,  81,  32, 388,  96, 146, 352,
       196, 358,  83, 483, 291, 317, 472, 314,  95, 476,  69, 415, 414,
       239, 326, 461, 111, 288,  80,  78, 453, 101,  63,  26, 22

In [71]:
# Word least representative of this topic
single_topic[98]

0.9512572321521888

In [72]:
# Word most representative of this topic
single_topic[50]

0.9734794751895403

In [73]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([349, 302, 112, 454, 273, 393, 375,  73, 484,  45])

In [74]:
top_word_indices = single_topic.argsort()[-10:]

In [75]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

recent
people
day
use
mother
son
security
carry
’
attack


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [76]:
for index,topic in enumerate(lda_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['stop', 'hospital', 'official', 'die', 'individual', 'recent', 'people', 'day', 'use', 'mother', 'son', 'security', 'carry', '’', 'attack']


THE TOP 15 WORDS FOR TOPIC #1
['community', 'house', 'official', 'company', '—', 'year ago', '’', 'like', 'speak', 'water', 'problem', 'live', 'year', 'ago', 'home']


THE TOP 15 WORDS FOR TOPIC #2
['want', 'know', 'kill', 'begin', '2', 'use', 'change', 'issue', 'people', 'police', 'thing', '’', 'justice', 'city', 'department']


THE TOP 15 WORDS FOR TOPIC #3
['way', 'percent', '—', 'family', 'song', 'feel', 'new', 'key', 'white', 'love', 'vote', '’', 'people', 'come', 'black']


THE TOP 15 WORDS FOR TOPIC #4
['poor', 'pay', '—', 'year', 'lead', '’', 'person', 's.', 'u. s.', 'people', 'spend', 'care', 'u.', 'country', 'health']


THE TOP 15 WORDS FOR TOPIC #5
['medium', '—', 'report', 'department', 'accord', 'case', 'law', 'person', 'statement', '’', 'white', 'news', 'police', 'woman', 'people']


THE TOP 15 WORDS F

### How to see the dominant topic in each document?
To classify a document as belonging to a particular topic, a logical approach is to see which topic has the highest contribution to that document and assign it.

In [35]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(dtm)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

NameError: name 'data' is not defined

### Review topics distribution across documents

In [ ]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, dtm, vectorizer, mds='tsne')
panel

### Get the top 15 keywords each topic

In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

### Attaching Discovered Topic Labels to Original Articles

In [21]:
dtm

<11992x17335 sparse matrix of type '<class 'numpy.int64'>'
	with 2635570 stored elements in Compressed Sparse Row format>

In [22]:
dtm.shape

(11992, 17335)

In [23]:
len(npr)

11992

In [24]:
topic_results = lda_model.transform(dtm)

In [25]:
topic_results.shape

(11992, 20)

In [26]:
topic_results[0]

array([7.46451900e-01, 8.10372795e-05, 8.10372785e-05, 4.42342109e-03,
       8.10372784e-05, 8.10372792e-05, 3.99263409e-02, 8.10372784e-05,
       4.17548848e-02, 8.10372787e-05, 4.65712745e-02, 8.10372782e-05,
       8.10372799e-05, 8.10372792e-05, 8.10372780e-05, 8.10372776e-05,
       5.54831933e-02, 6.02876444e-02, 4.12889383e-03, 8.10372790e-05])

In [27]:
topic_results[0].round(2)

array([0.75, 0.  , 0.  , 0.  , 0.  , 0.  , 0.04, 0.  , 0.04, 0.  , 0.05,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.06, 0.06, 0.  , 0.  ])

In [28]:
topic_results[0].argmax()

0

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [29]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [30]:
topic_results.argmax(axis=1)

array([ 0,  0,  0, ..., 17, 10, 17])

In [31]:
npr['Topic'] = topic_results.argmax(axis=1)

In [32]:
npr.head(10)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",0
1,Donald Trump has used Twitter — his prefe...,0
2,Donald Trump is unabashedly praising Russian...,0
3,"Updated at 2:50 p. m. ET, Russian President Vl...",0
4,"From photography, illustration and video, to d...",6
5,I did not want to join yoga class. I hated tho...,17
6,With a who has publicly supported the debunk...,11
7,"I was standing by the airport exit, debating w...",5
8,"If movies were trying to be more realistic, pe...",5
9,"Eighteen years ago, on New Year’s Eve, David F...",5


## Great work!

### How to GridSearch the best LDA model?
The most important tuning parameter for LDA models is n_components (number of topics). In addition, I am going to search learning_decay (which controls the learning rate) as well.

Besides these, other possible search params could be learning_offset (downweigh early iterations. Should be > 1) and max_iter. These could be worth experimenting if you have enough computing resources.

Be warned, the grid search constructs multiple LDA models for all possible combinations of param values in the param_grid dict. So, this process can consume a lot of time and resources.

In [33]:
from sklearn.model_selection import GridSearchCV


# Define Search Param
search_params = {'n_components': [15, 20, 25],'learning_decay': [.5, .7]}

# Init the Model
lda = LatentDirichletAllocation(max_iter=25)

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params, cv=3, verbose=2, n_jobs = -1)

# Do the Grid Search
model.fit(dtm)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  18 | elapsed:  2.5min remaining:  6.4min
[Parallel(n_jobs=-1)]: Done  15 out of  18 | elapsed:  2.5min remaining:   29.5s


KeyboardInterrupt: 

How to see the best topic model and its parameters?


In [ ]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

### Compare LDA Model Performance Scores

In [ ]:
# Get Log Likelyhoods from Grid Search Output
n_topics = [15, 20, 25]
log_likelyhoods_5 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.5]
log_likelyhoods_7 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.7]

# Show graph
plt.figure(figsize=(12, 8))
plt.plot(n_topics, log_likelyhoods_5, label='0.5')
plt.plot(n_topics, log_likelyhoods_7, label='0.7')
plt.title("Choosing Optimal LDA Model")
plt.xlabel("Num Topics")
plt.ylabel("Log Likelyhood Scores")
plt.legend(title='Learning decay', loc='best')
plt.show()

https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/

In [83]:
from sklearn.externals import joblib 
# Load the model from the file 
lda_model___ = joblib.load('saved_lda_model.pkl')  

/Users/gabrielseemann/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
